# Job Reassigment Problem with QAOA:
## Small instances and GPU running

In this notebook, we will experiment with the small instances generated by "random_instances_generation.ipynb". A simulation for QAOA optimization will be done for each instance. First, using a personal notebook CPU. Then, using google colab GPU.

The goals of the notebooks are:
* trying QAOA with small JRP instances that follows the heuristic rules (mentione in "random_instances_generation.ipynb").
* Achieving, at least, regular bistring measurements distributions for the QAOA optimizations. That is, not focusing on having excelent results, but that we are going in a good path.
* Comparing the simulation speedup between the personal notebook CPU vs. google colab GPU. This will be helpful for future experiments we will be working with bigger and more important instances.

For running the experiments, using a local envirioment and the google colab will be necessary.

## For Google Colab

The GPU experiment part will make use of Google Colab. Please only only follow the next instructions in case you are going to use Google Colab $\textbf{NOW}$. Doing the following in your local envirioment will install in your own PC openqaoa an other libraries in a local mode, which is not recomended. If you are going to keep using the local envirioment until the GPU experiment requires you to migrate to Google Colab, jump to the next section and came back here when the instructions tells you.

In [1]:
# 1. RUN THIS
!git clone https://github.com/entropicalabs/openqaoa.git

Cloning into 'openqaoa'...
remote: Enumerating objects: 12606, done.
remote: Counting objects: 100% (2485/2485), done.
remote: Compressing objects: 100% (667/667), done.
remote: Total 12606 (delta 2006), reused 2057 (delta 1806), pack-reused 10121
Receiving objects: 100% (12606/12606), 21.65 MiB | 20.90 MiB/s, done.
Resolving deltas: 100% (9361/9361), done.


2. Refresh the google colab files. Go to openqaoa/Makefile and replace this file with one of the following link (todavia no hice el link).

3. Go to openqaoa/src/openqaoa-core/requirements.txt and replace this file with the one of the following link (todavia no hice el link)

In [2]:
#4. RUN THIS
!cd openqaoa && make local-install

pip install ./src/openqaoa-core
Processing ./src/openqaoa-core
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 633.5/633.5 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 67.0 MB/s eta 0:00:00
  Created wheel for openqaoa-core: filename=openqaoa_core-0.2.5-py3-none-any.whl size=288132 sha256=0643acee6d97decef784a04aaee26e99fc3a674c611a79de5bfd99febefeca78
  Stored in directory: /root/.cache/pip/wheels/7e/1b/fd/0059597011958f73d116029f8bca232f17a9edffd00b5cd9b1
  Created wheel for docplex: filename=docplex-2.25.236-py3-none-any.whl size=671350 sha256=01581e6e513f5309a5152036146c56f8a6dad07d53da09cf5d3bb4d4151eb3da
  Stored in directory: /root/.cache/pip/wheels/3

## Importing the problem instances

In [3]:
import json
from openqaoa import QUBO
#from openqaoa.backends import create_device
from openqaoa import QAOA
from openqaoa.algorithms import QAOAResult

from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import CplexOptimizer, GurobiOptimizer

import timeit

ModuleNotFoundError: No module named 'qiskit_optimization'

In [ ]:
instances_indexes = range(5)
instances = []
for index in instances_indexes:
     with open('./instance%s/instance%s.json'%(str(index),str(index)), "r") as file:
         instances.append(json.load(file))
#instances[0]

In [ ]:
keys = list(instances[0].keys())
#keys

## Creating the QUBO formulation

In [ ]:
%run ../functions/qubo_for_jrp.ipynb
%run ../functions/qubo_in_openqaoa_format.ipynb

qubos = []
qubos_openqaoaformat = []
for inst in instances:
    qubo = qubo_for_jrp(inst)
    qubos.append(qubo)
    qubo = qubo_in_openqaoa_format(qubo)
    qubos_openqaoaformat.append(qubo)
#qubos[0]
#qubos_openqaoaformat[0]

## Using CplexOptimizer from Qiskit Optimization for classical solving

Explicar un poco de qiskit optimization y cplex

In [ ]:
print('----------------------------------------------------------------------------------------------')
for index,qubo in enumerate(qubos):
    num_variables = instances[index]['num_agents'] * instances[index]['num_vacnJobs']

    linear = {clave: qubo[clave] for clave in list(qubo.keys())[:num_variables]}
    quad = {clave: qubo[clave] for clave in list(qubo.keys())[num_variables:]}
    #print(linear)
    #print(quad)

    mod = QuadraticProgram("JRP")
    for i in  instances[index]['allBinaryVariables']:
        mod.binary_var(name="x"+str(i))

    mod.minimize(constant=3, linear=linear, quadratic=quad)
    #print(mod.prettyprint())
    result = CplexOptimizer().solve(mod)

    print('INSTANCE ',index,'\n')
    print(result.prettyprint())

    resultstring =""
    for val in result.variables_dict.values():
        resultstring += str(int(val))

    print('Solution : ',resultstring,'\n')
    print('----------------------------------------------------------------------------------------------')

----------------------------------------------------------------------------------------------
INSTANCE  0 

objective function value: -0.9199999999999999
variable values: x0=0.0, x1=0.0, x2=0.0, x3=0.0, x4=0.0, x5=0.0, x6=0.0, x7=0.0, x8=0.0, x9=0.0, x10=0.0, x11=1.0, x12=0.0, x13=0.0, x14=0.0
status: SUCCESS
Solution :  000000000001000 

----------------------------------------------------------------------------------------------
INSTANCE  1 

objective function value: -0.94
variable values: x0=0.0, x1=0.0, x2=0.0, x3=0.0, x4=0.0, x5=1.0, x6=0.0, x7=1.0, x8=0.0, x9=0.0, x10=0.0, x11=0.0, x12=0.0, x13=0.0, x14=0.0, x15=0.0, x16=0.0, x17=0.0
status: SUCCESS
Solution :  000001010000000000 

----------------------------------------------------------------------------------------------
INSTANCE  2 

objective function value: -2.1500000000000004
variable values: x0=0.0, x1=0.0, x2=0.0, x3=0.0, x4=0.0, x5=0.0, x6=0.0, x7=0.0, x8=0.0, x9=1.0, x10=0.0, x11=0.0, x12=1.0, x13=0.0, x14=0.0, x15

## Creating the Ising formulation for OpenQAOA

In [ ]:
isings = []
for index,qubo in enumerate(qubos_openqaoaformat):
    ising= QUBO.convert_qubo_to_ising(len(instances[index]['allBinaryVariables']), qubo[0], qubo[1])
    isings.append(ising)
#isings[0]

Please notice that, in OpenQAOA library, it is called to a QUBO problem to a $\textbf{ising formulation}$ of the problem. That means, with the binary variables domain $\{-1,1\}$. In OpenQAOA documentation, this is because they consider it as a $\textbf{QUBO formulation with an Ising encryption}$.

In [ ]:
openqaoa_qubos = []
for index,ising in enumerate(isings):
    openqaoa_qubo = QUBO(n = len(instances[index]['allBinaryVariables']), terms=ising[0], weights=ising[1])
    openqaoa_qubos.append(openqaoa_qubo)

## Using OpenQAOA brute force solver

OpenQAOA recommend bounding the brute force solver for <25 qubits problems.  Instance 4 has 30 variables. That's why we only search with this solver the solutions of the first four instances. The solutions given by the code below this markdown were:

* instance 0: Ground State energy: -3.9200000000000017, Solution: ['000000000001000']

* instance 1: Ground State energy: -3.9399999999999977, Solution: ['000001010000000000']

* instance 2: Ground State energy: -5.150000000000006, Solution: ['00000000010010000000']

* instance 3: Ground State energy: -2.8200000000000074, Solution: ['0000001000000001000000000']

In [ ]:
for openqaoa_qubo in openqaoa_qubos:
    solver = QAOA()
    solver.compile(openqaoa_qubo)
    solver.solve_brute_force(verbose=True)
    print()

## Running QAOA - personal notebook CPU

For running in a persona notebook CPU, we will limit to just using the first 3 instances, which also are the smallest ones. Instance 3 and 4 need 25 and 30 qubits respectively. These would result in a too big execution time, which goes out the aim of this notebook.

In [ ]:
#QAOA results and optimization execution times for each pair (instance,maxfev)
maxfevs = [1,2,3,4,5]
for index,qubo in enumerate(openqaoa_qubos[0:3]):
    execution_result={}
    for maxfev in maxfevs:
        execution_result['maxfev '+str(maxfev)] = {}

        q = QAOA()
        device = create_device(location='local', name='qiskit.shot_simulator')
        q.set_device(device)
        q.set_circuit_properties(p=15, param_type='standard', init_type='ramp', mixer_hamiltonian='x')
        q.set_backend_properties(n_shots=10000,prepend_state=None, append_state=None)
        q.set_classical_optimizer(method='nelder-mead', maxiter=500, tol=0.001,maxfev=maxfev,
                          optimization_progress=False, cost_progress=True, parameter_log=False)
        q.compile(qubo)

        start_time = timeit.default_timer()
        q.optimize()
        execution_time = timeit.default_timer() - start_time
        print('exe time for instance ',index,' and maxfev ',maxfev,': ',execution_time)

        execution_result['maxfev '+str(maxfev)]['time'] = execution_time
        execution_result['maxfev '+str(maxfev)]['result'] = q.result.asdict()
    with open('instance%s/personalNotebook.json'%(str(index)), 'w') as file:
        json.dump(execution_result, file)

exe time for instance  0  and maxfev  1 :  1.4891044000105467
exe time for instance  0  and maxfev  2 :  2.8193327999906614
exe time for instance  0  and maxfev  3 :  4.3221340000018245
exe time for instance  0  and maxfev  4 :  5.8232392000063555
exe time for instance  0  and maxfev  5 :  7.514105099995504
exe time for instance  1  and maxfev  1 :  2.9517350000096485
exe time for instance  1  and maxfev  2 :  5.958301499995287
exe time for instance  1  and maxfev  3 :  9.05619079999451
exe time for instance  1  and maxfev  4 :  12.379670700000133
exe time for instance  1  and maxfev  5 :  15.301291700001457
exe time for instance  2  and maxfev  1 :  4.820644999999786
exe time for instance  2  and maxfev  2 :  9.31256819999544
exe time for instance  2  and maxfev  3 :  13.767930000001797
exe time for instance  2  and maxfev  4 :  19.15205180000339
exe time for instance  2  and maxfev  5 :  23.597164200007683
